In [11]:
# import psycopg2
from datetime import date, datetime, timedelta
from twilio.rest import Client
import time
from datetime import datetime, timedelta

import os
from dotenv import load_dotenv
load_dotenv('./.env')

from sqlalchemy import create_engine
import pandas as pd

In [2]:
# PostgreSQL connection details
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_PORT = os.getenv("DB_PORT")

In [5]:
# Twilio account details
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_PHONE_NUMBER = os.getenv("TWILIO_PHONE_NUMBER")
TWILIO_MESSAGING_SERVICE_SID = os.getenv("TWILIO_MESSAGING_SERVICE_SID")

In [3]:
# Create the SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [4]:
# Create a DataFrame from the Calendar table
df_calendar = pd.read_sql_query('SELECT "UserId","Event","Description","DateStartEvent","HoursStartEvent","DateEndEvent","HoursEndEvent","Notify","StatoEvento","EventId" FROM "public"."Calendar"', engine)
df_calendar = df_calendar.rename(columns={'UserId': 'UUID'})
# display(df_calendar)
# Create a DataFrame from the AspNetUsers table
df_aspnetusers = pd.read_sql_query('SELECT "Id", "UserId" FROM "public"."AspNetUsers"', engine)
df_aspnetusers = df_aspnetusers.rename(columns={"Id":"UUID"})
# display(df_aspnetusers)

# Perform left join
df_calendar = pd.merge(df_calendar, df_aspnetusers, on='UUID', how='left')

# Create a DataFrame from the Users table
df_users = pd.read_sql_query('SELECT "Id","Surname","Name", "MobilePhone", "Email", "StatusUser", "Notify" FROM "public"."Users"', engine)
df_users = df_users.rename(columns={"Id":"UserId", "Notify":"Notification"})

# Perform left join
df_calendar = pd.merge(df_calendar, df_users, on='UserId', how='left')

df_calendar["DateStartEvent"] = pd.to_datetime(df_calendar["DateStartEvent"])  # Convert to datetime if not already done

# Convert 'HoursStartEvent' to string representation
df_calendar["HoursStartEvent"] = df_calendar["HoursStartEvent"].apply(lambda x: x.strftime('%H:%M:%S'))

# Concatenate 'DateStartEvent' and 'HoursStartEvent'
df_calendar["DateTimeStartEvent"] = pd.to_datetime(df_calendar["DateStartEvent"].astype(str) + ' ' + df_calendar["HoursStartEvent"])

# new column 'AdjustedDateTime' containing the adjusted datetime values
df_calendar["AdjustedDateTime"] = df_calendar["DateTimeStartEvent"] - df_calendar["Notify"]

display(df_calendar.head(5))


,UUID,Event,Description,DateStartEvent,HoursStartEvent,DateEndEvent,HoursEndEvent,Notify,StatoEvento,EventId,UserId,Surname,Name,MobilePhone,Email,StatusUser,Notification,DateTimeStartEvent,AdjustedDateTime
0,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,FAKE 2,[modes]Shared_scooter[wp]Cheapest,2023-07-03,12:45:25,2023-07-03,13:45:25,0 days 00:05:00,Created,ed9b89b8-d4f4-4fb3-a497-6f28e2f31566,74,faggian,giulio,3924162864,g.faggian@presstoday.com,Active,True,2023-07-03 12:45:25,2023-07-03 12:40:25
1,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,ll,"[modes]Bicycle,Train[wp]Ecofriendly",2023-07-03,12:48:36,2023-07-03,13:48:36,0 days 00:00:00,Created,ec82d246-cb78-4fbb-89a7-22b0ae94a3a9,74,faggian,giulio,3924162864,g.faggian@presstoday.com,Active,True,2023-07-03 12:48:36,2023-07-03 12:48:36
2,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,pippo,[modes]Shared_scooter[wp]Cheapest,2023-07-03,12:56:32,2023-07-03,13:56:32,0 days 00:05:00,Created,485bc9b8-8a29-4938-9140-47a1c2982706,74,faggian,giulio,3924162864,g.faggian@presstoday.com,Active,True,2023-07-03 12:56:32,2023-07-03 12:51:32
3,d5afa28c-8bf6-4ca1-9059-72298ad83e87,TestError,"[modes]Walk,Scooter,Subway[wp]Fastest",2023-07-20,14:50:19,2023-07-20,15:50:19,0 days 00:05:00,Created,8ca90450-e419-4350-bda8-17e034ca88cd,186,Cardozo,Nicolò,3469886132,n.cardozo@presstoday.com,Active,False,2023-07-20 14:50:19,2023-07-20 14:45:19
4,d5afa28c-8bf6-4ca1-9059-72298ad83e87,TestError,"[modes]Walk,Scooter,Subway[wp]Fastest",2023-07-20,14:50:19,2023-07-20,15:50:19,0 days 00:05:00,Created,a2fd71e8-80fe-4572-aa1b-14a0125d872a,186,Cardozo,Nicolò,3469886132,n.cardozo@presstoday.com,Active,False,2023-07-20 14:50:19,2023-07-20 14:45:19


In [6]:
import re
# Function to check and add country code
def add_country_code(phone_number):
    if re.match(r"^\+\d{1,}", phone_number):
        return phone_number  # Country code already present
    else:
        return "+" + phone_number  # Add country code

# Function to convert to ISO 8601 format
def convert_to_iso8601(datetime_str):
    # # Convert string to datetime object
    # datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    
    # Convert datetime object to ISO 8601 format
    iso8601_str = datetime_str.strftime("%Y-%m-%dT%H:%M:%S")
    
    return iso8601_str



In [7]:
# Apply the function to the "MobilePhone" column
df_calendar["MobilePhone"] = df_calendar["MobilePhone"].apply(add_country_code)

# Apply the function to the "AdjustedDateTime" column
df_calendar["AdjustedDateTime"] = df_calendar["AdjustedDateTime"].apply(convert_to_iso8601)
display(df_calendar.head(10))

,UUID,Event,Description,DateStartEvent,HoursStartEvent,DateEndEvent,HoursEndEvent,Notify,StatoEvento,EventId,UserId,Surname,Name,MobilePhone,Email,StatusUser,Notification,DateTimeStartEvent,AdjustedDateTime
0,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,FAKE 2,[modes]Shared_scooter[wp]Cheapest,2023-07-03,12:45:25,2023-07-03,13:45:25,0 days 00:05:00,Created,ed9b89b8-d4f4-4fb3-a497-6f28e2f31566,74,faggian,giulio,+3924162864,g.faggian@presstoday.com,Active,True,2023-07-03 12:45:25,2023-07-03T12:40:25
1,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,ll,"[modes]Bicycle,Train[wp]Ecofriendly",2023-07-03,12:48:36,2023-07-03,13:48:36,0 days 00:00:00,Created,ec82d246-cb78-4fbb-89a7-22b0ae94a3a9,74,faggian,giulio,+3924162864,g.faggian@presstoday.com,Active,True,2023-07-03 12:48:36,2023-07-03T12:48:36
2,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,pippo,[modes]Shared_scooter[wp]Cheapest,2023-07-03,12:56:32,2023-07-03,13:56:32,0 days 00:05:00,Created,485bc9b8-8a29-4938-9140-47a1c2982706,74,faggian,giulio,+3924162864,g.faggian@presstoday.com,Active,True,2023-07-03 12:56:32,2023-07-03T12:51:32
3,d5afa28c-8bf6-4ca1-9059-72298ad83e87,TestError,"[modes]Walk,Scooter,Subway[wp]Fastest",2023-07-20,14:50:19,2023-07-20,15:50:19,0 days 00:05:00,Created,8ca90450-e419-4350-bda8-17e034ca88cd,186,Cardozo,Nicolò,+3469886132,n.cardozo@presstoday.com,Active,False,2023-07-20 14:50:19,2023-07-20T14:45:19
4,d5afa28c-8bf6-4ca1-9059-72298ad83e87,TestError,"[modes]Walk,Scooter,Subway[wp]Fastest",2023-07-20,14:50:19,2023-07-20,15:50:19,0 days 00:05:00,Created,a2fd71e8-80fe-4572-aa1b-14a0125d872a,186,Cardozo,Nicolò,+3469886132,n.cardozo@presstoday.com,Active,False,2023-07-20 14:50:19,2023-07-20T14:45:19
5,d5afa28c-8bf6-4ca1-9059-72298ad83e87,TestErr,"[modes]Walk,Scooter,Subway[wp]Cheapest",2023-07-13,14:50:19,2023-07-13,15:50:19,0 days 00:05:00,Created,97cf3df2-3cc8-4e76-b06d-8dcf0a9b20bd,186,Cardozo,Nicolò,+3469886132,n.cardozo@presstoday.com,Active,False,2023-07-13 14:50:19,2023-07-13T14:45:19
6,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,pppp,[modes]Shared_scooter[wp]Cheapest,2023-07-08,14:48:14,2023-07-08,15:48:14,0 days 00:30:00,Created,827bc531-3c44-4ef3-bc61-50eb150403b5,74,faggian,giulio,+3924162864,g.faggian@presstoday.com,Active,True,2023-07-08 14:48:14,2023-07-08T14:18:14
7,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,Garibaldi meeting,,2023-06-30,10:57:22,2023-06-30,11:57:22,0 days 00:05:00,Cancelled,3ccfe868-5aec-4049-801a-4b9c515c34fa,74,faggian,giulio,+3924162864,g.faggian@presstoday.com,Active,True,2023-06-30 10:57:22,2023-06-30T10:52:22
8,d5afa28c-8bf6-4ca1-9059-72298ad83e87,test,"[modes]Walk,Scooter,Subway[wp]Fastest",2023-07-19,14:53:33,2023-07-19,15:53:33,0 days 00:05:00,Created,f1045088-0fe4-45f8-a64b-5108187feb7c,186,Cardozo,Nicolò,+3469886132,n.cardozo@presstoday.com,Active,False,2023-07-19 14:53:33,2023-07-19T14:48:33
9,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,prova,,2023-07-01,13:14:18,2023-07-01,14:14:18,0 days 00:30:00,Created,d75d5309-9151-4234-84c2-f2558d11f8d0,74,faggian,giulio,+3924162864,g.faggian@presstoday.com,Active,True,2023-07-01 13:14:18,2023-07-01T12:44:18


In [8]:
df = df_calendar[:2].copy()
df["MobilePhone"] = "+393206767511"
df["AdjustedDateTime"] = "2023-07-04T16:05:00"
df["Event"] = "This is a notification from AVTI"

df

,UUID,Event,Description,DateStartEvent,HoursStartEvent,DateEndEvent,HoursEndEvent,Notify,StatoEvento,EventId,UserId,Surname,Name,MobilePhone,Email,StatusUser,Notification,DateTimeStartEvent,AdjustedDateTime
0,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,This is a notification from AVTI,[modes]Shared_scooter[wp]Cheapest,2023-07-03,12:45:25,2023-07-03,13:45:25,0 days 00:05:00,Created,ed9b89b8-d4f4-4fb3-a497-6f28e2f31566,74,faggian,giulio,+393206767511,g.faggian@presstoday.com,Active,True,2023-07-03 12:45:25,2023-07-04T16:05:00
1,5ebdb461-4d27-49ca-a6a1-4b2c0b00c1a5,This is a notification from AVTI,"[modes]Bicycle,Train[wp]Ecofriendly",2023-07-03,12:48:36,2023-07-03,13:48:36,0 days 00:00:00,Created,ec82d246-cb78-4fbb-89a7-22b0ae94a3a9,74,faggian,giulio,+393206767511,g.faggian@presstoday.com,Active,True,2023-07-03 12:48:36,2023-07-04T16:05:00


In [9]:
df["AdjustedDateTime"].dtype

dtype('O')

In [12]:
# Create a Twilio client
client = Client(TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN)

# Function to send scheduled WhatsApp messages
def send_scheduled_whatsapp(row):
    phone_number = row["MobilePhone"]
    send_at = datetime.strptime(row["AdjustedDateTime"], "%Y-%m-%dT%H:%M:%S")
    notification_enabled = row["Notification"]
    event = row["Event"]
    print(phone_number)
    print(send_at)

    if notification_enabled:
        try:
            message = client.messages.create(
                body=event,
                to=f'whatsapp:{phone_number}',
                messaging_service_sid=TWILIO_MESSAGING_SERVICE_SID,
                send_at=send_at,
                schedule_type='fixed',
            )
            print(message.sid)
            print(f"Sending the following message: {message.body} to the following number: {phone_number}")
        except Exception as e:
            print(f"An error occurred while sending the message to following UUID({row['UUID']}) with following EventId({row['EventId']}): {str(e)}")
    else:
        print(f"Skipping notification for {phone_number} as 'Notification' is False.")



In [ ]:
# Apply the function to each row of the DataFrame
df.apply(send_scheduled_whatsapp, axis=1)
